In [ ]:
## Montar google drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Cargar el archivo desde Google Drive
import pandas as pd
# Cargar datos de entrenamiento y prueba
df_train = pd.read_excel("linkXlsx")
df_test = pd.read_excel("linkXlsx")



**Entrenamiento con el conjunto de Train**

Se seleccionan las caracteristicas y se buscan los mejores hiperparametros

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import joblib

# 1. Cargar archivo de entrenamiento

# 2. Seleccionar características
columnas_seleccionadas = [
    'POS','NEU','NEG',
   'alegria','tristeza' ,'miedo', 'disgusto', 'enojo', 'sorpresa','toxicity',
   'yo', 'me', 'mi','mí',
   'num_palabras_largas',
   'falta de motivacion',  'aislamiento social',  'pensamientos suicidas',
]

X_train = df_train[columnas_seleccionadas].copy()
y_train = df_train['label']

# 3. Normalizar columnas numéricas
scaler = MinMaxScaler()
X_train[['num_palabras_largas']] = scaler.fit_transform(X_train[[ 'num_palabras_largas']])


#Hiperparametros encontrados en el conjutno de pruebas
modelo_rf = RandomForestClassifier(class_weight= 'balanced', max_depth= 15, min_samples_leaf= 2, min_samples_split= 5, n_estimators= 100, random_state=42)



modelo_rf.fit(X_train, y_train)

# 8. Guardar el modelo, escalador, label encoder y columnas seleccionadas
joblib.dump(modelo_rf, 'modelo_rf_entrenado.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(columnas_seleccionadas, 'columnas_seleccionadas.pkl')

print("Modelo entrenado y guardado exitosamente.")


In [ ]:
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 2. Cargar modelo, scaler, encoder y columnas seleccionadas
modelo_rf = joblib.load('modelo_rf_entrenado.pkl')
scaler = joblib.load('scaler.pkl')
columnas_seleccionadas = joblib.load('columnas_seleccionadas.pkl')

# 3. Preprocesar el conjunto de prueba
X_test = df_test.reindex(columns=columnas_seleccionadas, fill_value=0)
X_test.loc[:, ['num_palabras_largas']] = scaler.transform(
    X_test[[ 'num_palabras_largas']]
)
y_test = df_test['label']

# 4. Hacer predicciones en el conjunto de prueba
y_pred = modelo_rf.predict(X_test)

# 5. Calcular métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Resultados finales en el conjunto de prueba:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))


**Distribucion de clases**

In [ ]:
# Comparar distribución de clases
print("Distribución de clases en Train:")
print(df_train['label'].value_counts(normalize=True))

print("\nDistribución de clases en Test:")
print(df_test['label'].value_counts(normalize=True))


In [ ]:
# Comparar distribuciones de una característica
import matplotlib.pyplot as plt

for col in columnas_seleccionadas:
    plt.figure(figsize=(6, 4))
    df_train[col].hist(alpha=0.5, color='blue', label='Train')
    df_test[col].hist(alpha=0.5, color='red', label='Test')
    plt.legend()
    plt.title(f'Distribución de {col} en Train y Test')
    plt.show()
